In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ylila\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ylila\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ylila\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ylila\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Preprocessing sentences

In [3]:
df = pd.read_csv('Data/labeled_sentences_merged_imbalanced_12k_extra_classes.csv',usecols=['sentence','topic_label','sentiment_label'])


In [4]:
X = df[['sentence']].astype(str)
y_topic = df.loc[:,'topic_label'].values
y_sentiment = df.loc[:,'sentiment_label'].values

# Build vectors

In [5]:
stop_words = set(stopwords.words('english')) 

In [6]:
bow_vectorizer = CountVectorizer(stop_words=stop_words,ngram_range=(1,3))
X_bow = bow_vectorizer.fit_transform(X.sentence)
X_bow

<12139x193940 sparse matrix of type '<class 'numpy.int64'>'
	with 494612 stored elements in Compressed Sparse Row format>

In [7]:
tfidf_vectorizer = TfidfVectorizer(,stop_words=stop_words,ngram_range=(1,3))
X_tfidf = tfidf_vectorizer.fit_transform(X.sentence)
X_tfidf

<12139x193940 sparse matrix of type '<class 'numpy.float64'>'
	with 494612 stored elements in Compressed Sparse Row format>

# Topic classifier BoW-(1,3)-gram

In [12]:
model = MultinomialNB()
cv_topic_nb_bow = cross_validate(model, X_bow, y_topic,cv=10, scoring = ('accuracy','precision_macro','recall_macro','f1_macro'))
for key in cv_topic_nb_bow:
    print(f'{key} : {cv_topic_nb_bow[key].mean()}')

fit_time : 0.09991827011108398
score_time : 0.036967015266418456
test_accuracy : 0.6915807065412996
test_precision_macro : 0.5238457563629889
test_recall_macro : 0.5581469200600161
test_f1_macro : 0.5141230085811612


# Sentiment Classifier BoW-(1,3)-gram

In [13]:
model = MultinomialNB()
cv_sentiment_nb_bow = cross_validate(model, X_bow, y_sentiment,cv=10, scoring = ('accuracy','precision_macro','recall_macro','f1_macro'))
for key in cv_sentiment_nb_bow:
    print(f'{key} : {cv_sentiment_nb_bow[key].mean()}')

fit_time : 0.06337838172912598
score_time : 0.025641989707946778
test_accuracy : 0.5940493636347586
test_precision_macro : 0.5259497844410499
test_recall_macro : 0.6563064194715615
test_f1_macro : 0.5139707274528267


# Topic Classifier tfidf-(1,3)-gram

In [14]:
model = MultinomialNB()
cv_topic_nb_tfidf = cross_validate(model, X_tfidf, y_topic,cv=10, scoring = ('accuracy','precision_macro','recall_macro','f1_macro'))
for key in cv_topic_nb_tfidf:
    print(f'{key} : {cv_topic_nb_tfidf[key].mean()}')

c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\U

fit_time : 0.0807800054550171
score_time : 0.03146758079528809
test_accuracy : 0.6605183955800084
test_precision_macro : 0.48785825687709694
test_recall_macro : 0.31044944670188135
test_f1_macro : 0.2974636711049059


c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Sentiment Classifier tfidf-(1,3)-gram

In [15]:
model = MultinomialNB()
cv_sentiment_nb_tfidf = cross_validate(model, X_tfidf, y_sentiment,cv=10, scoring = ('accuracy','precision_macro','recall_macro','f1_macro'))
for key in cv_sentiment_nb_tfidf:
    print(f'{key} : {cv_sentiment_nb_tfidf[key].mean()}')

c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\U

fit_time : 0.0752028465270996
score_time : 0.0286731481552124
test_accuracy : 0.7763406044620945
test_precision_macro : 0.4061987900607553
test_recall_macro : 0.3402838625199016
test_f1_macro : 0.3042770479212063


c:\Users\ylila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
